In [58]:
import torch
from torch import nn

from torchvision import datasets
from torchvision.transforms import ToTensor, ToPILImage, Grayscale
from torch.utils.data import DataLoader, Subset
import torchvision.models as models

In [81]:
from torchvision.models import resnet50, MobileNet_V2_Weights

In [135]:
model = models.mobilenet_v2(pretrained=True).to(device);model


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [136]:
labels = ('T-shirt/top',
'Trouser',
'Pullover',
'Dress',
'Coat',
'Sandal',
'Shirt',
'Sneaker',
'Bag',
'Ankle boot');

In [137]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [138]:
weights = MobileNet_V2_Weights.DEFAULT
preprocess = weights.transforms(); preprocess


ImageClassification(
    crop_size=[224]
    resize_size=[232]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [139]:
def tfms(img):
    img = Grayscale(num_output_channels=3)(img)
    img = ToTensor()(img)
    return img

In [140]:
# transform from [1, 28, 28] => [3, 28, 28] https://pytorch.org/vision/stable/generated/torchvision.transforms.Grayscale

train_dataset = datasets.FashionMNIST(root='data', train=True, transform=tfms, download=True)
test_dataset = datasets.FashionMNIST(root='data', train=False, transform=tfms, download=True)

In [141]:
img = train_dataset[0][0];img

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]])

In [142]:
datasets.FashionMNIST?

Init signature:
datasets.FashionMNIST(
    root: str,
    train: bool = True,
    transform: Optional[Callable] = None,
    target_transform: Optional[Callable] = None,
    download: bool = False,
) -> None
Docstring:     
`Fashion-MNIST <https://github.com/zalandoresearch/fashion-mnist>`_ Dataset.

Args:
    root (string): Root directory of dataset where ``FashionMNIST/raw/train-images-idx3-ubyte``
        and  ``FashionMNIST/raw/t10k-images-idx3-ubyte`` exist.
    train (bool, optional): If True, creates dataset from ``train-images-idx3-ubyte``,
        otherwise from ``t10k-images-idx3-ubyte``.
    download (bool, optional): If True, downloads the dataset from the internet and
        puts it in root directory. If dataset is already downloaded, it is not
        downloaded again.
    transform (callable, optional): A function/transform that  takes in an PIL image
        and returns a transformed version. E.g, ``transforms.RandomCrop``
    target_transform (callable, optional): A fu

In [143]:
batch = preprocess(img).unsqueeze(0)


In [144]:
batch.shape

torch.Size([1, 3, 224, 224])

In [145]:
prediction = model(batch)


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [147]:
prediction.squeeze(0).shape

torch.Size([10])

In [148]:
p = prediction.squeeze(0).softmax(0)

In [149]:
model.classifier

Sequential(
  (0): Dropout(p=0.2, inplace=False)
  (1): Linear(in_features=1280, out_features=1000, bias=True)
)

In [150]:
p[p.argmax(0)]

tensor(0.1744, grad_fn=<SelectBackward0>)

In [157]:
model.classifier[1] = nn.Linear(1280, 10).to(device)

In [158]:
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

In [159]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=1e-3)

In [160]:
def train(model, dataloader, loss_fn, optimizer):
    model.train()
    num_batches = len(dataloader)
    loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        logits = model(X)
        loss = loss_fn(logits, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    print(f'training loss: {loss}')

def test(model, dataloader, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    loss, correct = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)
            preds = model(X)
            loss += loss_fn(preds, y)
            correct += (preds.argmax(dim=1) == y).type(torch.float).sum().item()
    correct /= size
    loss /= num_batches
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")
    return (correct, loss)


In [161]:
train_dataloader = DataLoader(train_dataset, batch_size=100)
test_dataloader = DataLoader(test_dataset, batch_size=100)

In [162]:
for _ in range(10):
    train(model, train_dataloader, loss_fn, optimizer)
    test(model, test_dataloader, loss_fn)

training loss: 0.989644467830658
Test Error: 
 Accuracy: 69.4%, Avg loss: 0.884394 

training loss: 1.0034676790237427
Test Error: 
 Accuracy: 70.5%, Avg loss: 0.848740 

training loss: 1.0172085762023926
Test Error: 
 Accuracy: 71.0%, Avg loss: 0.828547 

training loss: 0.9089383482933044
Test Error: 
 Accuracy: 71.0%, Avg loss: 0.823982 

training loss: 1.003945231437683
Test Error: 
 Accuracy: 70.7%, Avg loss: 0.827744 

training loss: 0.9269333481788635
Test Error: 
 Accuracy: 71.4%, Avg loss: 0.816538 

training loss: 1.0019218921661377
Test Error: 
 Accuracy: 71.4%, Avg loss: 0.815662 

training loss: 0.9161587357521057
Test Error: 
 Accuracy: 71.4%, Avg loss: 0.816561 

training loss: 0.9906447529792786
Test Error: 
 Accuracy: 71.5%, Avg loss: 0.814629 

training loss: 0.9556320309638977
Test Error: 
 Accuracy: 71.9%, Avg loss: 0.801296 

